<a href="https://colab.research.google.com/github/AlinaZemlyakova/CompLing-2021-2022/blob/main/hw10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 10. Генерация текста

### Задание 1 (8 баллов).

Попробуйте дообучать GPT на каком-то другом тексте (можете попробовать любые стихи или какие-то специфичные вещи вроде анекдотов, теорий заговоров, постов в помоечных телеграм каналах, текстов журналистов и СМИ с выразительным стилем). 
Попробуйте разные методы и параметры генерации (beam search, температура, top_k и тп). Сохраните в тетрадке несколько хороших сгенерированных текстов


In [ ]:
!pip install transformers
!pip install datasets
import re
import datasets
from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling
import torch
DEVICE = torch.device("cuda:0")

     |████████████████████████████████| 4.2 MB 8.4 MB/s 
     |████████████████████████████████| 596 kB 69.3 MB/s 
     |████████████████████████████████| 6.6 MB 48.9 MB/s 
     |████████████████████████████████| 84 kB 4.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 346 kB 6.9 MB/s 
     |████████████████████████████████| 212 kB 52.9 MB/s 
     |████████████████████████████████| 1.1 MB 61.1 MB/s 
     |████████████████████████████████| 140 kB 55.6 MB/s 
     |████████████████████████████████| 127 kB 76.7 MB/s 
     |████████████████████████████████| 271 kB 73.9 MB/s 
     |████████████████████████████████| 94 kB 4.6 MB/s 
     |████████████████████████████████| 144 kB 79.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ER

Загружаем модель ruGPT-small от Сбера

In [ ]:
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

В качестве обучающего датасета был взят датасет с песнями Бориса Гребенщикова, спарсенными с сайта Genuis. В примере песни "25 к 10" и "212-85-06".

In [ ]:
dataset = load_dataset("huggingartists/boris-grebenshikov")

Using custom data configuration default


Generating train split: 0 examples [00:00, ? examples/s]

Dataset lyrics_dataset downloaded and prepared to /root/.cache/huggingface/datasets/huggingartists___lyrics_dataset/default/1.0.0/d9837ea53ef7618273cf54c0e01ea29efdb882770a70a51284e11159a46f3335. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset['train']['text'][3:5]

['Я инженер на сотню рублей\nИ больше я не получу\nМне двадцать пять, и я до сих пор\nНе знаю, чего хочу\nИ мне кажется, нет никаких оснований\nГордиться своей судьбой\nНо если б я мог выбирать себя\nЯ снова бы стал собой\nМне двадцать пять, и десять из них\nЯ пою, не зная, о чем\nИ мне так сложно бояться той\nЧто стоит за левым плечом\nИ пускай мои слова не ясны\nВ этом мало моей вины\nНо что до той, что стоит за плечом\nПеред нею мы все равны\nМожет статься, что завтра стрелки часов\nНачнут вращаться назад\nИ тот, кого с плачем снимали с креста\nОкажется вновь распят\nИ нежные губы станут опять\nИскать своего Христа\nНо я пел, что пел, и хотя бы в том\nСовесть моя чиста\nЯ счастлив тем, как сложилось все\nДаже тем, что было не так\nДаже тем, что ветер в моей голове\nИ в храме моем бардак\nЯ просто пытался растить свой сад\nИ не портить прекрасный вид\nИ начальник заставы поймет меня\nИ беспечный рыбак простит',
 'Если бы я знал, что такое электричество\nЯ сделал бы шаг, я вышел на ул

Чистим тексты от переносов строк и лишних символов и сохраняем обучающие данные в .txt файл.

In [ ]:
songs_clear = [re.sub('\n|\u2005', ' ', i) for i in dataset['train']['text']]

In [ ]:
train_path = open('train_dataset.txt', 'w')
for s in songs_clear:
    train_path.write(s + '\n')

In [ ]:
train_path = 'train_dataset.txt'

# Создание датасета
train_dataset = TextDataset( tokenizer=tokenizer,file_path=train_path,block_size=64, 
                            overwrite_cache=True)
  
# специальный класс который будет подавать в модель данные в нужном ей виде
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


Обучаем модель.

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments( 
    output_dir= "./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=100, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32,  
    gradient_accumulation_steps=16, 
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 1538
  Num Epochs = 100
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 16
  Total optimization steps = 300


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=300, training_loss=3.7271122233072917, metrics={'train_runtime': 2543.3265, 'train_samples_per_second': 60.472, 'train_steps_per_second': 0.118, 'total_flos': 5023273992192000.0, 'train_loss': 3.7271122233072917, 'epoch': 99.98})

При генерации текстов были протестированы различные методы и параметры генерации (beam search, температура, top_k и тп). В результате сильнее всего влиял парамет температуры: повышение до 1.4 дало самый адекватный результат (при 1.0 было много повторов, при 1.6 текст был слишком хаотичен).

In [ ]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=1.4,
                        top_k=50,
                        max_length=100,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Дождь идет  Но его эхо еще далече Так откуда мы знаем твой голос? Ты идешь за мной И с каждым днем свет все ближе И свет все ближе И, хотя внутри пустота И ты мне не брат Теперь ты мой друг Теперь время жить для меня На берегу моря и на острове Но нет мне дороги назад По этой траве мне некуда идти Ты идешь за мной В своей чистой, лесной траве И не знаешь, кто я И никто уже никуда от меня не ждет Никто уже не возьмет твоих рук И тебе


In [ ]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=1.4,
                        beam_search=5,
                        max_length=100,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Дождь идет  А я хочу быть один на траве Где-то в тайге А в небе только ночь - только листопад Теперь я всегда уверен: На земле еще есть другие берега Чтобы уйти в тень Неси мою почту наверх - Все, что сказано внизу То уже не важно, что ты хочешь в завтра На самом деле мне нужно больше Так, что мне делать с ним дальше Но если бы не его я, никто бы меня Не заметил Мне кажется, что он все правильно подметил Когда сказал:


In [ ]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=1.4,
                        beam_search=5,
                        top_k=15,
                        max_length=100,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Дождь идет 
Здравствуй, небо ясное И вечный дождь Над твоей головой Невидимый, но грозящий Заставить меня петь Я люблю тебя, я знаю, ты скажешь это — Когда я скажу это я смогу дышать Не зная, о чем я. Не знающий, чего хочет Он станет моим домом, он станет моей тенью Он не станет слушать меня Он не уйдет, пока ты хочешь Не глядя ни на что Я люблю тебя Но мне кажется, небо не смотрит Я всегда знал, что ты


In [ ]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=1.4,
                        beam_search=5,
                        top_k=80,
                        max_length=100,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Дождь идет  Ночью, когда все спят; в этом тихом месте Мне показалось Я чувствую знакомый рок Снаружи, все тихо В сумерках мне слышится плач И только в лесу, за высокой вершиной Меня не тронут ничьи молитвы Вокруг нас таинственный лес Деревьев, но каждый шаг — шорох Где-то живет птица — все больше горькая, чем грустная Птицы танцуют под звонкую дробь Оставив в покое свои ветви Здесь не видно стекла Но здесь есть следы Кто-


### Задание  2 (2 балла)

Ответьте на следующие вопросы:

1) В каких статья были представлены GPT-1, GPT-2, GPT-3?

2) Как собирался обучающий корпус для GPT-3? Каким образом создатели старались обеспечить высокое качество текстов в обучающей выборке?

**Ответы:**

1) GPT-1 была представлена в статьев Improving Language Understanding by Generative Pre-Training (Radford, et al., 2018), GPT-2 - в статье Language Models are Unsupervised Multitask Learners (Radford, et al., 2019), GPT-3 - в Language Models are Few-Shot Learners (Brown, et al., 2020)

2) Обучающий корпус для GPT-3 состоял из 600 гигабайтов русскоязычных текстов: Common Crawl, WebText2, 2 книжных корпуса и Википедия. Создатели старались обеспечить высокое качество текстов в обучающей выборке и улучшить общее качество модели тем, что более качественным частям корпусов был приписан больший вес при обучении, и отрывки из них использовались чаще. Например, корпус Common Crawl имел вес 60%, WebText2 - 22%, остальные - менее 10%.
